In [1]:
# importing all the libraries needed 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
sns.set_style("whitegrid")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error

#For date time functions
from datetime import datetime
from datetime import timedelta
import math

# Importing the most popular regression libraries.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, ridge_regression, Lasso, SGDRegressor, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
train_data=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip',parse_dates=True)
sample_submission=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip')
features_data=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip',parse_dates=True)
stores_data=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
test_data=pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')

In [ ]:
from fbprophet import Prophet
import tqdm.notebook as tq
df = pd.DataFrame()
for i in tq.tqdm(range(1,46)):
    model=Prophet()
    filled=features_data[((features_data['Store']==i) & (features_data['Date']<'2013-05-03'))][['Date','CPI']]
    tserie = filled.rename(columns = {'Date': 'ds', 'CPI': 'y'}, inplace = False)
    tserie =tserie.sort_values(by=['ds'])
    tserie['ds'] = pd.to_datetime(tserie['ds'])
    model.fit(tserie)
    future_dates=model.make_future_dataframe(periods=13,freq = 'W',include_history =True)
    future_dates['ds'] = future_dates['ds'].apply(lambda x: x + timedelta(days=5))
    prediction=model.predict(future_dates)
    df=df.append(prediction)

df.reset_index(drop=True)
features_data['CPI1']=np.nan
for i,j in enumerate(df['yhat']):
    features_data['CPI1'].iloc[i]=j

features_data=features_data.drop(['CPI'],axis=1)
features_data = features_data.rename(columns = {'CPI1': 'CPI'})

In [ ]:
from fbprophet import Prophet
import tqdm.notebook as tq
df = pd.DataFrame()
for i in tq.tqdm(range(1,46)):
    model=Prophet()
    filled=features_data[((features_data['Store']==i) & (features_data['Date']<'2013-05-03'))][['Date','Unemployment']]
    tserie = filled.rename(columns = {'Date': 'ds', 'Unemployment': 'y'}, inplace = False)
    tserie =tserie.sort_values(by=['ds'])
    tserie['ds'] = pd.to_datetime(tserie['ds'])
    model.fit(tserie)
    future_dates=model.make_future_dataframe(periods=13,freq = 'W',include_history =True)
    future_dates['ds'] = future_dates['ds'].apply(lambda x: x + timedelta(days=5))
    prediction=model.predict(future_dates)
    df=df.append(prediction)

df.reset_index(drop=True)
features_data['Unemployment1']=np.nan
for i,j in enumerate(df['yhat']):
    features_data['Unemployment1'].iloc[i]=j

features_data=features_data.drop(['Unemployment'],axis=1)
features_data = features_data.rename(columns = {'Unemployment1': 'Unemployment'})

In [ ]:
stores = stores_data.merge(features_data, on ='Store' , how = 'left')
final_data_train = train_data.merge(stores, on = ['Store', 'Date', 'IsHoliday'], how = 'left')

In [ ]:
stores = stores_data.merge(features_data, on ='Store' , how = 'left')
final_data_test = test_data.merge(stores, on = ['Store', 'Date', 'IsHoliday'], how = 'left')

In [ ]:
print(final_data_train.shape)
print(final_data_test.shape)

In [ ]:
def markdown_imputation(final_data):    
    final_data.loc[final_data.MarkDown1.isnull() ,'MarkDown1']= 0
    final_data.loc[final_data.MarkDown2.isnull() ,'MarkDown2']= 0
    final_data.loc[final_data.MarkDown3.isnull() ,'MarkDown3']= 0
    final_data.loc[final_data.MarkDown4.isnull() ,'MarkDown4']= 0
    final_data.loc[final_data.MarkDown5.isnull() ,'MarkDown5']= 0
    return final_data

In [ ]:
def weekly_sales_imputation(final_data):    
    final_data[final_data.Weekly_Sales<0]
    print('before removing negative weekly sales ,shape of final dataframe is :',final_data.shape)
    final_data=final_data[final_data.Weekly_Sales>=0]
    print('after removing negative weekly sales ,shape of final dataframe is :',final_data.shape)
    return final_data

In [ ]:
def train_temp_bins(final_data):
    temp_100_110_f=final_data[((final_data.Temperature>100) & (final_data.Temperature< 110))].Weekly_Sales.sum()
    temp_90_100_f=final_data[((final_data.Temperature>90) & (final_data.Temperature< 100))].Weekly_Sales.sum()
    temp_80_90_f=final_data[((final_data.Temperature>80) & (final_data.Temperature< 90))].Weekly_Sales.sum()
    temp_70_80_f=final_data[((final_data.Temperature>70) & (final_data.Temperature< 80))].Weekly_Sales.sum()
    temp_60_70_f=final_data[((final_data.Temperature>60) & (final_data.Temperature< 70))].Weekly_Sales.sum()
    temp_50_60_f=final_data[((final_data.Temperature>50) & (final_data.Temperature< 60))].Weekly_Sales.sum()
    temp_40_50_f=final_data[((final_data.Temperature>40) & (final_data.Temperature< 50))].Weekly_Sales.sum()
    temp_30_40_f=final_data[((final_data.Temperature>30) & (final_data.Temperature< 40))].Weekly_Sales.sum() 
    temp_0_30_f=final_data[((final_data.Temperature>0) & (final_data.Temperature< 30))].Weekly_Sales.sum()   
    temp_less_than_0_f=final_data[((final_data.Temperature>-10) & (final_data.Temperature< 0))].Weekly_Sales.sum()
    final_data['Temp_bins'] = np.nan
    final_data.loc[((final_data.Temperature>-10) & (final_data.Temperature<0)) ,'Temp_bins']= temp_less_than_0_f
    final_data.loc[((final_data.Temperature>0) & (final_data.Temperature< 30)) ,'Temp_bins']= temp_0_30_f
    final_data.loc[((final_data.Temperature>30) & (final_data.Temperature< 40)) ,'Temp_bins']= temp_30_40_f
    final_data.loc[((final_data.Temperature>40) & (final_data.Temperature< 50)) ,'Temp_bins']= temp_40_50_f
    final_data.loc[((final_data.Temperature>50) & (final_data.Temperature< 60)) ,'Temp_bins']= temp_50_60_f
    final_data.loc[((final_data.Temperature>60) & (final_data.Temperature< 70)) ,'Temp_bins']= temp_60_70_f
    final_data.loc[((final_data.Temperature>70) & (final_data.Temperature< 80)) ,'Temp_bins']= temp_70_80_f
    final_data.loc[((final_data.Temperature>80) & (final_data.Temperature< 90)) ,'Temp_bins']= temp_80_90_f
    final_data.loc[((final_data.Temperature>90) & (final_data.Temperature< 100)),'Temp_bins']= temp_90_100_f
    final_data.loc[((final_data.Temperature>100) & (final_data.Temperature< 110)),'Temp_bins']= temp_100_110_f
    final_data.loc[final_data.Temp_bins.isnull() ,'Temp_bins']= 0
    list1=[temp_less_than_0_f,temp_0_30_f,temp_30_40_f,temp_40_50_f,temp_50_60_f,temp_60_70_f,temp_70_80_f,temp_80_90_f,temp_90_100_f,temp_100_110_f]
    return final_data,list1

In [ ]:
def test_temp_bins(final_data,list1):
    final_data['Temp_bins'] = np.nan
    final_data.loc[((final_data.Temperature>-10) & (final_data.Temperature<0)) ,'Temp_bins']= list1[0]
    final_data.loc[((final_data.Temperature>0) & (final_data.Temperature< 30)) ,'Temp_bins']= list1[1]
    final_data.loc[((final_data.Temperature>30) & (final_data.Temperature< 40)) ,'Temp_bins']= list1[2]
    final_data.loc[((final_data.Temperature>40) & (final_data.Temperature< 50)) ,'Temp_bins']= list1[3]
    final_data.loc[((final_data.Temperature>50) & (final_data.Temperature< 60)) ,'Temp_bins']= list1[4]
    final_data.loc[((final_data.Temperature>60) & (final_data.Temperature< 70)) ,'Temp_bins']= list1[5]
    final_data.loc[((final_data.Temperature>70) & (final_data.Temperature< 80)) ,'Temp_bins']= list1[6]
    final_data.loc[((final_data.Temperature>80) & (final_data.Temperature< 90)) ,'Temp_bins']= list1[7]
    final_data.loc[((final_data.Temperature>90) & (final_data.Temperature< 100)),'Temp_bins']= list1[8]
    final_data.loc[((final_data.Temperature>100) & (final_data.Temperature< 110)),'Temp_bins']= list1[9]
    final_data.loc[final_data.Temp_bins.isnull() ,'Temp_bins']= 0
    return final_data

In [ ]:
def split(final_data):
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    final_data['Year'] = final_data['Date'].dt.year
    final_data['Month']= final_data['Date'].dt.month
    final_data['Week'] = final_data['Date'].dt.week
    final_data['Day']  = final_data['Date'].dt.day
    return final_data

In [ ]:
import datetime
def days_from_christmas_for_train(x):
    if x['Year']== 2010 :
        diff=datetime.datetime(2010, 12, 31)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 12, 30))):
        diff=datetime.datetime(2011, 12, 30)-x['Date']
        return diff.days
    else:
        return 0

In [ ]:
import datetime
def days_from_christmas_for_test(x):
    if x['Year']== 2010 :
        diff=datetime.datetime(2010, 12, 31)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 12, 30))):
        diff=datetime.datetime(2011, 12, 30)-x['Date']
        return diff.days
    if ((x['Year']== 2012) and (x['Date']< datetime.datetime(2012, 12, 28))):
        diff=datetime.datetime(2012, 12, 28)-x['Date']
        return diff.days
    if ((x['Year']== 2013) and (x['Date']< datetime.datetime(2013, 12, 27))):
        diff=datetime.datetime(2013, 12, 27)-x['Date']
        return diff.days
    else:
        return 0   

In [ ]:
def days_from_thanksgiving_for_train(x):
    if ((x['Year']== 2010) and (x['Date']< datetime.datetime(2010, 11, 26))):
        diff=datetime.datetime(2010, 11, 26)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 11, 25))):
        diff=datetime.datetime(2011, 11, 25)-x['Date']
        return diff.days
    else:
        return 0

In [ ]:
def days_from_thanksgiving_for_test(x):
    if ((x['Year']== 2010) and (x['Date']< datetime.datetime(2010, 11, 26))):
        diff=datetime.datetime(2010, 11, 26)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 11, 25))):
        diff=datetime.datetime(2011, 11, 25)-x['Date']
        return diff.days
    if ((x['Year']== 2012) and (x['Date']< datetime.datetime(2012, 11, 23))):
        diff=datetime.datetime(2012, 11, 23)-x['Date']
        return diff.days
    if ((x['Year']== 2013) and (x['Date']< datetime.datetime(2013, 11, 29))):
        diff=datetime.datetime(2013, 11, 29)-x['Date']
        return diff.days
    else:
        return 0

In [ ]:
def holiday_type(x):
    if   (x['IsHoliday']== 1) & (x['Week']==6):
        return 1 #SuperBowl
    elif (x['IsHoliday']== 1) & (x['Week']==36):
        return 2 #LaborDay
    elif (x['IsHoliday']== 1) & (x['Week']==47):
        return 3 #Thanksgiving
    elif (x['IsHoliday']== 1) & (x['Week']==52):
        return 4 #Christmas
    else:
        return 0

In [ ]:
def holiday_label(final_data):
    final_data.loc[(final_data.IsHoliday==True) ,'IsHoliday']= 1
    final_data.loc[(final_data.IsHoliday==False) ,'IsHoliday']= 0
    return final_data

In [ ]:
def type_label(final_data):
    final_data.loc[(final_data.Type=='A') ,'Type']= 1
    final_data.loc[(final_data.Type=='B') ,'Type']= 2
    final_data.loc[(final_data.Type=='C') ,'Type']= 3
    return final_data

In [ ]:
import holidays
def holiday_in_week_train(final_data):
    dates =[]
    for ptr in holidays.US(years = 2010).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2011).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2012).items():
        dates.append(ptr[0])
    holiday_count=[] 
    for index, row in final_data.iterrows():
        dat = final_data['Date'][index]
        dt=[]
        for i in range(0,5):
            dt.append(dat - datetime.timedelta(days = i))
        for i in range(1,3):
            dt.append(dat + datetime.timedelta(days = i))
        count = 0
        for date in dates:
            if date in dt:
                count +=1
        holiday_count.append(count)
    return holiday_count  

In [ ]:
import holidays
def holiday_in_week_test(final_data):
    dates =[]
    for ptr in holidays.US(years = 2010).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2011).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2012).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2013).items():
        dates.append(ptr[0])
    holiday_count=[] 
    for index, row in final_data.iterrows():
        dat = final_data['Date'][index]
        dt=[]
        for i in range(0,5):
            dt.append(dat - datetime.timedelta(days = i))
        for i in range(1,3):
            dt.append(dat + datetime.timedelta(days = i))
        count = 0
        for date in dates:
            if date in dt:
                count +=1
        holiday_count.append(count)
    return holiday_count

## Training data preprocessing and generating new features

In [ ]:
final_data_train=markdown_imputation(final_data_train)
final_data_train=weekly_sales_imputation(final_data_train)
final_data_train,list1=train_temp_bins(final_data_train)
final_data_train=split(final_data_train)
final_data_train['diff_from_christmas'] = final_data_train.apply(days_from_christmas_for_train, axis=1)  
final_data_train['days_from_thanksgiving'] = final_data_train.apply(days_from_thanksgiving_for_train, axis=1)  
final_data_train['IsHoliday_bins'] = final_data_train.apply(holiday_type, axis=1)
final_data_train=holiday_label(final_data_train)
final_data_train=type_label(final_data_train)
final_data_train['Holidays'] = np.array(holiday_in_week_train(final_data_train))

## Test data preprocessing and generating new features

In [ ]:
final_data_test=markdown_imputation(final_data_test)
final_data_test=test_temp_bins(final_data_test,list1)
final_data_test=split(final_data_test)
final_data_test['diff_from_christmas'] = final_data_test.apply(days_from_christmas_for_test, axis=1) 
final_data_test['days_from_thanksgiving'] = final_data_test.apply(days_from_thanksgiving_for_test, axis=1)   
final_data_test['IsHoliday_bins'] = final_data_test.apply(holiday_type, axis=1)
final_data_test=holiday_label(final_data_test)
final_data_test=type_label(final_data_test)
final_data_test['Holidays'] = np.array(holiday_in_week_test(final_data_test))

In [ ]:
final_data_train=final_data_train.reset_index(drop=True)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
final_data_train=final_data_train[['Store','Dept','IsHoliday','Size','Week','Type','Year','Weekly_Sales','Holidays','Day']]
final_data_test=final_data_test[['Store','Dept','IsHoliday','Size','Week','Type','Year','Holidays','Day']]
# final_data_train['Date']=pd.to_numeric(pd.to_datetime(final_data_train['Date']))
# final_data_test['Date'] = pd.to_numeric(pd.to_datetime(final_data_test['Date']))
final_data_train['IsHoliday']=final_data_train['IsHoliday'].astype('bool')
final_data_test['IsHoliday']=final_data_test['IsHoliday'].astype('bool')
final_data_train['Type']=final_data_train['Type'].astype('int')
final_data_test['Type']=final_data_test['Type'].astype('int')
y = final_data_train['Weekly_Sales']
X = final_data_train.drop(['Weekly_Sales'], axis=1)
rf_Model = RandomForestRegressor(n_estimators= 140,max_depth=27,n_jobs = -1)
rf_Model.fit(X, y)
y_hat= rf_Model.predict(final_data_test)

In [ ]:
sample_submission['Weekly_Sales']  = list(y_hat)

In [ ]:
sample_submission.to_csv('submission.csv',index = False)